## 基于word counts特征，使用xgboost：

In [17]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline

In [18]:
import jieba
#处理训练集，将训练集的文本信息和label信息合并，清洗特殊符号，同时将文本内容进行分词
def merge_feature_label(feature_name,label_name):
    feature=pd.read_csv(feature_name,sep=",")
    label=pd.read_csv(label_name,sep=",")
    data=feature.merge(label,on='id')
    data["X"]=data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    # 丢掉缺失值
    dataDropNa=data.dropna(axis=0, how='any')
    print(dataDropNa.info())
    dataDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    dataDropNa["X_split"]=dataDropNa["X"].apply(lambda x:" ".join(jieba.cut(x)))
    return dataDropNa
 
dataDropNa=merge_feature_label("Train_DataSet.csv","Train_DataSet_Label.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7265 entries, 0 to 7339
Data columns (total 5 columns):
id         7265 non-null object
title      7265 non-null object
content    7265 non-null object
label      7265 non-null int64
X          7265 non-null object
dtypes: int64(1), object(4)
memory usage: 340.5+ KB
None


C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    # apply()方法能劫持另外一个对象的方法，继承另外一个对象的属性 
    # lambda https://blog.csdn.net/zjuxsl/article/details/79437563
    #  join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    # 列名信息
    print(test.info())
    # replace(old, new) 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次。
    test["X"]=test["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    # jieba.cut()按照词性切分“我爱中国”切分成我爱中国
    test["X_split"]=test["X"].apply(lambda x:" ".join(jieba.cut(x)))
    return test
 
testData=process_test("Test_DataSet.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 4 columns):
id         7356 non-null object
title      7356 non-null object
content    7288 non-null object
X          7356 non-null object
dtypes: object(4)
memory usage: 230.0+ KB
None


In [20]:
train_x = dataDropNa.X_split.values # 训练集文本分词数据
train_y = dataDropNa.label.values # 训练集标签
test_x = testData.X_split.values #测试集文本分词数据

In [21]:
# 将数据分成训练和验证集。我们可以使用scikit-learn的model_selection模块中的train_test_split来完成它。
xtrain, xvalid, ytrain, yvalid = train_test_split(train_x, train_y,
                                                  stratify=train_y, 
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)

In [28]:
# 通过损失函数值来选择模型。
def multiclass_logloss(actual, predicted, eps = 1e-15):
    "对数损失度量"
    
    if len(actual.shape)==1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [22]:
print (xtrain.shape) 
print (xvalid.shape)

(6538,)
(727,)


使用词汇计数(Word Counts)作为功能。可以使用scikit-learn

In [23]:
# 向量化
ctv = CountVectorizer(min_df=3, 
                      max_df=0.5, 
                      ngram_range=(1,2)
                     )

使用Count Vectorizer来fit训练集中划分出来的的训练集文本分词数据xtrain和验证集文本分词数据xvalid，训练整个训练集文本分词数据train_x和测试集文本分词数据test_x

In [25]:
# Vectorizer来fit训练集中划分出来的的训练集文本分词数据xtrain和验证集文本分词数据xvalid
ctv.fit(list(xtrain) + list(xvalid)) 
xtrain_ctv = ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)


# Vectorizer来fit训练整个训练集文本分词数据traindata和测试集文本分词数据testdata
ctv.fit(list(train_x) + list(test_x)) 
train_x_ctv = ctv.transform(train_x)
test_x_ctv = ctv.transform(test_x)


利用提取的word counts特征来fit一个简单的Logistic Regression .

In [29]:
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial') 
clf.fit(xtrain_ctv, ytrain) 
predictions = clf.predict_proba(xvalid_ctv) 

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.600 


效果略差。。。

接下来,尝试一个非常简单的模型- 朴素贝叶斯。

利用提取的word counts特征来fitNaive Bayes 

In [31]:
clf = MultinomialNB() 
clf.fit(xtrain_ctv, ytrain) 
predictions = clf.predict_proba(xvalid_ctv) 
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 6.870 


效果不好。。。

基于word counts特征，使用xgboost：

In [33]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.590 


In [34]:
# 基于word counts特征的xgboost模型logloss最小，选择此模型，再进行总的训练和测试
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(train_x_ctv.tocsc(), train_y)
predictions = clf.predict_proba(test_x_ctv.tocsc())


In [39]:
predictions

array([[ 0.4990451 ,  0.48234281,  0.01861211],
       [ 0.09148925,  0.46422803,  0.44428271],
       [ 0.00578942,  0.84613031,  0.14808029],
       ..., 
       [ 0.11486442,  0.88370866,  0.00142692],
       [ 0.16532095,  0.82567328,  0.00900571],
       [ 0.07465193,  0.91159981,  0.01374823]], dtype=float32)

In [40]:
preds=np.argmax(predictions,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)


(7356, 1)


In [45]:
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('word_counts_xgboost.csv',index=None)